In [1]:
import numba
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [28]:
bok = 70
rozmiar = bok*bok
J = 1
B = 2
beta = 0.00001
l = 50 # liczba kroków symulacji
H0 = 0

In [29]:
model = np.random.randint(-1, 1, (bok, bok))
for i in range(bok):
    for j in range(bok):
        if (model[i,j] == 0):
            model[i,j] = 1
# plt.imshow(model)
# model

In [30]:
energiaPoj = 0
energiaRzedu = 0
energiaCalkowita = 0

# w poziomie
for x in range(bok):
    for e in range(bok):
        if (e < bok-1): 
            energiaPoj = -J*model[x, e-1]*model[x, e]
            energiaRzedu = energiaRzedu + energiaPoj
    energiaCalkowita = energiaCalkowita + energiaRzedu
    energiaRzedu = 0


# w pionie
for x in range(bok):
    for e in range(bok):
        if (x < bok-1): 
            energiaPoj = -J*model[x, e-1]*model[x, e]
            energiaRzedu = energiaRzedu + energiaPoj
    energiaCalkowita = energiaCalkowita + energiaRzedu
    energiaRzedu = 0


# wpływ pola magnetycznego
for e in range(rozmiar):
    energiaCalkowita = energiaCalkowita -B*e

H0 = energiaCalkowita
energiaCalkowita = 0

In [33]:
@numba.njit
def step(model, H0):

    for x in range(rozmiar*l):
        energiaRzedu = 0
        energiaCalkowita = 0

        rand = np.random.randint(0, bok-1)
        rand2 = np.random.randint(0, bok-1)
        spinka = model[rand, rand2]   # losowanie spinu

        if(spinka == 1): 
            spinka = -1
        else:
            spinka = 1

        model[rand, rand2] = spinka  # zamiana stanu na przeciwny


        # w poziomie
        for x in range(bok):
            for e in range(bok):
                if (e < bok-1): 
                    energiaPoj = -J*model[x, e-1]*model[x, e]
                    energiaRzedu = energiaRzedu + energiaPoj
            energiaCalkowita = energiaCalkowita + energiaRzedu
            energiaRzedu = 0


        # w pionie
        for x in range(bok):
            for e in range(bok):
                if (x < bok-1): 
                    energiaPoj = -J*model[x, e-1]*model[x, e]
                    energiaRzedu = energiaRzedu + energiaPoj
            energiaCalkowita = energiaCalkowita + energiaRzedu
            energiaRzedu = 0


        for e in numba.prange(rozmiar):
            energiaCalkowita = energiaCalkowita -B*e

        H1 = energiaCalkowita
        energiaCalkowita = 0


        deltaH = H1 - H0

        H0 = H1


        # prawdopodobieństwo akceptacji dla dodatniej delty H
        if (deltaH > 0):
            P = np.exp(-beta*deltaH)
            r = np.random.randint(0, 1000)/1000
            if (P < r):
                model[rand, rand2] = -spinka

        #plt.imshow(model)
        #plt.show()

    return model, H0
    


model, H0 = step(model, H0)
#
# plt.imshow(model)



# buffer = np.zeros_like(model)

# fig = plt.figure(figsize = (10, 10))
# data = plt.imshow(model)

# plt.close()

# def frame(i):
#     global model, H0
#     model, H0 = step(model, H0)
#     #model, buffer = buffer, model
#     data.set_data(model)
#     return data, # przecinek - tuple z jednym elementem

# animation = FuncAnimation(fig, frame, frames = rozmiar*l, blit = True)
# HTML(animation.to_jshtml())